In [ ]:
'''
use .zip with texts
'''
import zipfile
import os
import shutil


with zipfile.ZipFile('100.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')



In [ ]:
'''
remove sample_data 
'''

shutil.rmtree('/content/sample_data')
os.remove('10.zip')

In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 8.2 MB 11.9 MB/s 


In [ ]:
from nltk.tokenize import sent_tokenize
import re
import pymorphy2
import argparse
import sys
import csv

morph = pymorphy2.MorphAnalyzer()

In [ ]:
# get pos

def get_pos(x):
    return morph.parse(x)[0].tag.POS


In [ ]:
# get normal form of word

def get_normal_form(word):
    return morph.parse(word)[0].normal_form

In [ ]:
# get sentences from text

def get_sentences(*args, **kvargs):
    return sent_tokenize(*args, **kvargs)


In [ ]:
# gwt words from text

def get_words(text):
    words = (
        re.sub(r'(^[^a-zA-Zа-яА-Я\d]+)|([^a-zA-Zа-яА-Я\d]+$)', '', word)
        for word in text.split())
    return tuple(word for word in words if word)


In [ ]:
# get normal form words list

def get_normalized_words(text):
    words = get_words(text)
    return tuple(get_normal_form(word) for word in words)


In [ ]:
# get symbol n-grams of word

def ngrams_letters_for_word(word, n):
    for i in range(len(word) - n + 1):
        return word[i:i + n]


In [ ]:
# get symbol n-grams of text

def ngrams_letters(text, n):
    words = get_words(text)
    output = list()
    for word in words:
        ngrams = ngrams_letters_for_word(word, n)
        if ngrams:
            output.append(ngrams)
    return output


In [ ]:
# get word n-grams of text

def ngrams_words(text, n):
    parts = re.split(r'[\.!?;]+', text)
    output = list()
    for part in parts:
        words = [morph.parse(x)[0].normal_form for x in get_normalized_words(part)]
        for i in range(len(words)-n+1):
            output.append(tuple(words[i:i+n]))
    return output

In [ ]:
# punct distribution

import nltk
nltk.download('punkt')

def get_punctuation_freq(text):
    punct = re.findall(r'[\.,!?;:]{1,6}', text)
    punct += re.findall(r'[\'\"\(\)]', text)
    punct += re.findall(r'\b\-+\b', text)
    counter = Counter(punct)
    common = counter.most_common(35)
    freq = {a[0]: a[1] / len(get_sentences(text)) for a in common}
    return freq


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# get the most common words n-gramms distribution

def get_common_ngrams_words_freq(text, n, count=350):
    ngrams_list = ngrams_words(text, n)
    counter = Counter(ngrams_list)
    common = counter.most_common(count)
    freq = {a[0]: a[1]/len(ngrams_list) for a in common}
    return freq

In [ ]:
# get the most common symbol n-gramms distribution

def get_common_ngrams_letters_freq(text, n, count=250):
    ngrams_list = ngrams_letters(text, n)
    counter = Counter(ngrams_list)
    common = counter.most_common(count)
    freq = {a[0]: a[1]/len(ngrams_list) for a in common}
    return freq


In [ ]:
# get pos distribution

def get_part_of_speech_freq(text):
    words = [get_pos(x) for x in get_words(text)]#if morph.parse(x)[0].tag.POS]
    counter = Counter(words)
    common = counter.most_common(20)
    freq = {a[0]: a[1] / len(words) for a in common}
    return freq

In [ ]:
# get average lenght of paragraph

def get_average_paragraph_length(text):
    return len(get_sentences(text)) / len(re.split(r'\n+', text))

In [ ]:
# get average word lenght

def get_average_word_length_in_letters(text):
    words = get_words(text)
    return sum(map(len, words)) / len(words)

In [ ]:
# get average sentence lenght

def get_average_sentense_length(text):
    return len(get_words(text)) / len(get_sentences(text))

In [ ]:
# character distribution

def get_chars_statistic(text):
    chars = re.findall(r'[a-яА-Я]', text)
    counter = Counter(chars)
    common = counter.most_common(33)
    freq = {a[0]:a[1]/len(chars) for a in common}
    return freq

In [ ]:
# minimax normalize

def normalize(data):
    norm_data = []
    for x in data:
        x = (x-min(data))/(max(data)-min(data))
        norm_data.append(x)
    return norm_data

In [ ]:
sharov_keys = ['в', 'быть', 'а', 'все', 'весь', 'бы', 'вот', 'время', 'видеть', 'без', 'во', 'вдруг', 'большой', 'взять', 'ведь', 'ваш', 'вода', 'все', 'больше', 'белый', 'вид', 'вечер', 'бог', 'вопрос', 'вернуться', 'взгляд', 'бояться', 'вместе', 'более', 'вообще', 'быстро', 'бывать', 'война', 'будто', 'бросить', 'будто', 'брать', 'войти', 'вести', 'воздух', 'верить', 'брат', 'бежать', 'ветер', 'берег', 'волос', 'видно', 'вещь', 'великий', 'впрочем', 'внимание', 'ведь', 'бутылка', 'возле', 'вздохнуть', 'близкий', 'бумага', 'вперед', 'вполне', 'вера', 'больше', 'вовсе', 'врач', 'бить', 'вокруг', 'вниз', 'баба', 'боль', 'вместо', 'вроде', 'возвращаться', 'власть', 'важный', 'водка', 'ах', 'бывший', 'возможность', 'бабушка', 'бок', 'видимо', 'век', 'веселый', 'висеть', 'возможно', 'взглянуть', 'вновь', 'военный', 'вон', 'бой', 'армия', 'вверх', 'враг', 'впервые', 'быстрый', 'воля', 'вокруг', 'б', 'виноватый', 'вино', 'волна', 'больница', 'броситься', 'впереди', 'весело', 'внимательно', 'возникнуть', 'воскликнуть', 'бедный', 'вроде', 'вагон', 'взяться', 'бегать', 'внезапно', 'больно', 'верный', 'восемь', 'внутренний', 'велеть', 'возраст', 'ворота', 'беда', 'вдоль', 'весна', 'болеть', 'автобус', 'бабка', 'вечный', 'болезнь', 'весьма', 'автор', 'бровь', 'будущее', 'больной', 'врать', 'билет', 'адрес', 'бросать', 'автомат', 'внизу', 'верно', 'вернуть', 'впечатление', 'буква', 'волноваться', 'больший', 'вероятно', 'видный', 'банк', 'борьба', 'автомобиль', 'воспоминание', 'возможный', 'водитель', 'верхний', 'боевой', 'ветка', 'беседа', 'больной', 'бледный', 'восторг', 'абсолютно', 'богатый', 'вкус', 'брюки', 'вряд', 'включить', 'бумажка', 'будущий', 'ага', 'близко', 'ботинок', 'вокзал', 'возразить', 'везти', 'возникать', 'английский', 'ведро', 'ближайший', 'банка', 'вина', 'валяться', 'борт', 'бандит', 'веревка', 'внутри', 'бродить', 'американский', 'бормотать', 'волк', 'во-первых', 'буквально', 'баня', 'борода', 'войско', 'вздрогнуть', 'водить', 'аккуратно', 'вернее', 'вот-вот', 'вдвоем', 'взрыв', 'бесконечный', 'взрослый', 'вечно', 'блестеть', 'бороться', 'биться', 'вариант', 'волнение', 'башня', 'вовремя', 'воевать', 'везде', 'большинство', 'видать', 'вечерний', 'барон', 'ванная', 'асфальт', 'белье', 'боец', 'аппарат', 'беспокоиться', 'внук', 'важно', 'бокал', 'бригада', 'во-вторых', 'божий', 'более', 'вершина', 'блестящий', 'артист', 'база', 'внутри', 'благородный', 'вообще-то', 'внешний', 'ворот', 'бык', 'академик', 'библиотека', 'болтать', 'было', 'вор', 'бочка', 'варить', 'влажный', 'бревно', 'балкон', 'вздыхать', 'видеться', 'вздох', 'батарея', 'благодарить', 'возмутиться', 'возвращение', 'возражать', 'безопасность', 'ангел', 'август', 'волшебный', 'а', 'вождь', 'воображение', 'восьмой', 'болото', 'беседовать', 'вежливо', 'бессмысленный', 'бомба', 'висок', 'базар', 'блеск', 'воскресенье', 'барак', 'бедро', 'возиться', 'воздушный', 'восток', 'воин', 'бросаться', 'благодаря', 'бар', 'веко', 'безумный', 'взрослый', 'воротник', 'бригадир', 'агент', 'актер', 'атака', 'внезапный', 'благодарность', 'буфет', 'вне', 'восточный', 'владеть', 'бред', 'беречь', 'анекдот', 'вглядываться', 'вздрагивать', 'вон', 'брак', 'американец', 'бизнес', 'босой', 'верх', 'воспользоваться', 'бабочка', 'влезть', 'взор', 'впрямь', 'вес', 'винтовка', 'арестовать', 'аэропорт', 'весенний', 'визит', 'волновать', 'будка', 'великолепный', 'вкусный', 'вначале', 'а-а', 'виднеться', 'воробей', 'башка', 'вправду', 'возить', 'большевик', 'буркнуть', 'браться', 'велосипед', 'взвод', 'владелец', 'братец', 'адвокат', 'аж', 'вопль', 'вдали', 'внутрь', 'бег', 'бумажный', 'восемьдесят', 'ванна', 'ветвь', 'береза', 'блюдо', 'вертолет', 'вертеться', 'бешеный', 'вмешаться', 'аккуратный', 'болтаться', 'бывало', 'ввести', 'взлететь', 'валить', 'валенок', 'впоследствии', 'беспокойство', 'влюбиться', 'виски', 'внешность', 'внимательный', 'верста', 'боковой', 'величество', 'витрина', 'беспокоить', 'ворваться', 'акт', 'вилка', 'академия', 'ахнуть', 'богатство', 'букет', 'бульвар', 'ад', 'батюшка', 'версия', 'ведать', 'водиться', 'восемнадцать', 'арест', 'благодарный', 'вдова', 'вольный', 'акция', 'бесконечно', 'бутерброд', 'виновато', 'веранда', 'аллея', 'влияние', 'авторитет', 'бодро', 'видать', 'внучка', 'бассейн', 'безусловно', 'бесполезный', 'вечность', 'воровать', 'внести', 'восхищение', 'вплотную', 'больничный', 'вертеть', 'воспринимать', 'восстановить', 'атмосфера', 'батальон', 'будить', 'ветерок', 'вой', 'брести', 'военный', 'включать', 'благо', 'близость', 'бетонный', 'бесшумно', 'вложить', 'ворона', 'видение', 'вручить', 'апрель', 'барышня', 'битва', 'брюхо', 'владыка', 'ай', 'аромат', 'бытие', 'блок', 'визг', 'абсолютный', 'армейский', 'блондинка', 'включая', 'бездна', 'бережно', 'ведьма', 'взорваться', 'воспитание', 'бегом', 'бодрый', 'бомж', 'вообразить', 'беззвучно', 'бензин', 'блокнот', 'бюро', 'возвращать', 'бинокль', 'бурный', 'безумие', 'бородатый', 'верховный', 'буря', 'величина', 'всадник', 'веселье', 'анализ', 'вежливый', 'вправо', 'бес', 'алый', 'актриса', 'блеснуть', 'бронзовый', 'ворчать', 'вредный', 'безнадежно', 'вонючий', 'воевода', 'архив', 'весть', 'временной', 'альбом', 'возбудить', 'бас', 'беспомощный', 'боярин', 'барабан', 'влево', 'волшебник', 'впереди', 'белеть', 'вена', 'весло', 'врезаться', 'англичанин', 'аппетит', 'акцент', 'аэродром', 'визжать']
uni_keys = ['а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'щ', 'ш', 'ъ', 'ь', 'э', 'ю', 'я']
pos_keys = ['NOUN', 'VERB', 'ADJF', 'PREP', 'CONJ', 'NPRO', 'ADVB', 'PRCL', 'INFN', 'GRND', 'PRTF', 'ADJS', 'NUMR', 'PRED', 'COMP', 'PRTS', 'INTJ']
punct_keys = ['!', '"', '!!', '!!!', ',', '-', '.', ':', ';', '?', '..', '...', ',.', '??', '???', '?!', '!?', '/']
tri_keys = ['арб', 'бед', 'без', 'бес', 'бла', 'бог', 'бол', 'бро', 'буд', 'был', 'быт', 'вел', 'вер', 'вес', 'веч', 'вид', 'вла', 'вно', 'воз', 'вой', 'вол', 'вос', 'вре', 'все', 'вст', 'вся', 'выс', 'где', 'гес', 'гла', 'гов', 'год', 'гол', 'гор', 'гос', 'гот', 'гре', 'дав', 'даж', 'дал', 'дев', 'дей', 'дел', 'ден', 'дер', 'для', 'дни', 'доб', 'дов', 'дол', 'дом', 'дос', 'дре', 'дру', 'дум', 'душ', 'евг', 'его', 'ему', 'есл', 'еще', 'жен', 'жиз', 'заб', 'зав', 'зак', 'зам', 'зан', 'зап', 'зас', 'зде', 'зем', 'зна', 'изв', 'или', 'име', 'имп', 'имя', 'иск', 'исп', 'ист', 'каж', 'каз', 'как', 'кам', 'ког', 'кол', 'ком', 'кон', 'кот', 'кра', 'лег', 'лет', 'лид', 'лиц', 'лиш', 'луч', 'люб', 'люд', 'мал', 'мат', 'мед', 'мен', 'мес', 'меч', 'мил', 'мин', 'мир', 'мне', 'мно', 'мог', 'мое', 'мож', 'мои', 'мой', 'мол', 'моя', 'над', 'наз', 'нак', 'нам', 'нап', 'нас', 'нач', 'наш', 'нев', 'нед', 'ней', 'нем', 'нео', 'неп', 'нес', 'нет', 'неу', 'ник', 'нич', 'нов', 'обе', 'обр', 'объ', 'оде', 'оди', 'одн', 'ока', 'она', 'они', 'опя', 'ост', 'отв', 'оте', 'отк', 'отц', 'оче', 'пер', 'пис', 'пла', 'поб', 'пов', 'под', 'поз', 'пок', 'пол', 'пом', 'пон', 'поп', 'пор', 'пос', 'пот', 'поч', 'пра', 'пре', 'при', 'про', 'пус', 'пут', 'раб', 'рад', 'раз', 'рас', 'реш', 'рим', 'род', 'рук', 'сам', 'све', 'сво', 'свя', 'сде', 'себ', 'сег', 'сем', 'сен', 'сер', 'сес', 'сил', 'ска', 'ско', 'сла', 'сле', 'сло', 'слу', 'сме', 'смо', 'соб', 'сов', 'сос', 'спо', 'спр', 'сре', 'ста', 'сто', 'стр', 'суд', 'сча', 'так', 'там', 'тво', 'теб', 'тем', 'теп', 'тог', 'тол', 'том', 'тор', 'тот', 'тре', 'три', 'уви', 'уда', 'уже', 'узн', 'ули', 'уме', 'уст', 'фла', 'хот', 'хоч', 'хра', 'хри', 'цел', 'час', 'чел', 'чем', 'чер', 'что', 'чув', 'эти', 'это', 'юни', 'юно']
bigr_keys = ['ав', 'ак', 'ал', 'ам', 'ан', 'ап', 'ар', 'ат', 'аф', 'ба', 'бе', 'би', 'бл', 'бо', 'бр', 'бу', 'бы', 'ва', 'вв', 'вд', 'ве', 'вз', 'ви', 'вк', 'вл', 'вм', 'вн', 'во', 'вп', 'вр', 'вс', 'вт', 'вх', 'вч', 'вы', 'га', 'гд', 'ге', 'ги', 'гл', 'гн', 'го', 'гр', 'гу', 'да', 'дв', 'де', 'ди', 'дл', 'дн', 'до', 'др', 'ду', 'ды', 'дя', 'ев', 'ег', 'ед', 'ее', 'еж', 'ей', 'ем', 'еп', 'ес', 'ех', 'ещ', 'жа', 'жд', 'же', 'жи', 'жр', 'за', 'зв', 'зд', 'зе', 'зи', 'зл', 'зм', 'зн', 'зо', 'зр', 'иб', 'иг', 'ид', 'из', 'ил', 'им', 'ин', 'ио', 'ис', 'ит', 'их', 'ищ', 'ка', 'кв', 'ке', 'ки', 'кл', 'кн', 'ко', 'кр', 'кт', 'ку', 'ла', 'ле', 'ли', 'ло', 'лу', 'ль', 'лю', 'ма', 'ме', 'ми', 'мн', 'мо', 'мр', 'му', 'мы', 'мя', 'на', 'не', 'ни', 'но', 'нр', 'ну', 'ны', 'об', 'ов', 'ог', 'од', 'ож', 'оз', 'ок', 'ол', 'он', 'оп', 'ор', 'ос', 'от', 'ох', 'оч', 'ош', 'па', 'пе', 'пи', 'пл', 'по', 'пр', 'пу', 'пы', 'пь', 'пя', 'ра', 'ре', 'ри', 'ро', 'ру', 'ры', 'ря', 'са', 'сб', 'св', 'сд', 'се', 'сж', 'си', 'ск', 'сл', 'см', 'сн', 'со', 'сп', 'ср', 'ст', 'су', 'сх', 'сц', 'сч', 'сы', 'сю', 'та', 'тв', 'те', 'ти', 'то', 'тр', 'ту', 'тщ', 'ты', 'тю', 'тя', 'уб', 'ув', 'уг', 'уд', 'уе', 'уж', 'уз', 'ук', 'ул', 'ум', 'ун', 'уп', 'ур', 'ус', 'ут', 'ух', 'уч', 'уш', 'фа', 'фе', 'фи', 'фл', 'фо', 'фр', 'ха', 'хв', 'хи', 'хл', 'хо', 'хр', 'ху', 'ца', 'цв', 'це', 'ци', 'ча', 'че', 'чи', 'чл', 'чт', 'чу', 'ша', 'ше', 'ши', 'шк', 'шл', 'шу', 'ще', 'эл', 'эн', 'эт', 'юл', 'юн', 'юп', 'яв', 'яз', 'яр', 'яс']


In [ ]:
# path to zip with authors

path = '/content/authors/content' 

# folders with texts

list_names = os.listdir(path)
print(list_names) 

roots = []
for name in list_names:
    roots.append(path+'/'+name)

print(roots)
label = 0 
for t_path in roots:
    label+=1
    for root, dirs, files in os.walk(t_path):
        for filename in files:
                file_path = t_path+'/'+filename
                with open(file_path, 'r', encoding='utf-8') as f:
                  print(file_path)
                  text = f.read().lower() # lower case 
                  normalized_words = get_normalized_words(text) # normal form 
                  sharov_row = [normalized_words.count(word) / len(normalized_words) for word in sharov_keys]
                  print(sharov_row) # frequencies distribution of words
                  uni_row = [text.count(word) / len(text) for word in sorted(uni_keys)]
                  print(uni_row) # unigramms distribution
                  punct_row = [text.count(word) / len(text) for word in sorted(punct_keys)]
                  print(punct_row) # punct marks distribution
                  tri_row = [text.count(word) / len(text) for word in sorted(tri_keys)]
                  print(tri_row) # 3-gramms frequency distribution
                  bigr_row = [text.count(word) / len(text) for word in sorted(bigr_keys)]
                  print(bigr_row) # 2-gramms frequency distribution
                  pos_row  = [get_pos(x) for x in get_words(text)]
                  pos = [pos_row.count(word) / len(pos_row) for word in sorted(pos_keys)]
                  print(pos) # pos distribution
                  # common vector
                  data = punct_row
                  for x in pos:
                    data.append(x)
                  for x in uni_row:
                    data.append(x)     
                  for x in bigr_row:    
                    data.append(x)
                  for x in tri_row:
                    data.append(x)
                  for x in sharov_row:
                    data.append(x)
                  final_data = normalize(data) # normalized vector
                  final_data.insert(0, label) # insert author's label
                  
                  # get csv for each author
                  with open(t_path + ' features'+'.csv', 'a', newline='') as csvfile: 
                    writer = csv.writer(csvfile)
  
                    writer.writerow(final_data)
                    csvfile.close()





['dostoewskij_f_m', 'nekrasow_n_a', 'esenin_s_a', 'blok_a_a', 'krylow_i_a']
['/content/5//dostoewskij_f_m', '/content/5//nekrasow_n_a', '/content/5//esenin_s_a', '/content/5//blok_a_a', '/content/5//krylow_i_a']
/content/5//dostoewskij_f_m/text_0070.shtml.txt


KeyboardInterrupt: ignored

In [ ]:
path = '/content/10/'

list_names = os.listdir(path)
print(list_names)
roots = []

for name in list_names:
    roots.append(path+'/'+name)

print(roots)

for t_path in roots:
    for root, dirs, files in os.walk(t_path):
        for filename in files:
                file_path = t_path+'/'+filename
                print(t_path+'/'+filename)              # path to current text
                with open (file_path, 'r') as f:
                  text = f.read()
                  fileID=0
                  threshold = 15000                     # threshold of text lenght
                  while threshold <= len(text):
                    with open(file_path + str(fileID) + '.txt','a', encoding='utf-8') as currentFile:
                      new_text = text[threshold-50000:threshold]
                      currentFile.write(new_text)
                      currentFile.close()
                      fileID+=1
                      threshold+=15000
                      
                      ''' 
                      this is the "tail" - what remains less than the threshold length,
                      there is no point in writing this to a separate file, since there may be just a couple of paragraphs of text

                  with open(file_path + str(fileID) + '.txt', 'a', encoding='utf-8') as currentFile:
                    new_text = text[threshold - 50000::]
                    currentFile.write(new_text)
                    currentFile.close()
                    '''
                os.remove(file_path)                      # remove text

['goncharow_i_a', 'kjuhelxbeker_w_k', 'grigorowich_d_w', 'dobroljubow_n_a', 'gorxkij_m', 'brjusow_w_j', 'gogolx_n_w', 'karamzin_n_m', 'esenin_s_a', 'grech_n_i']
['/content/10//goncharow_i_a', '/content/10//kjuhelxbeker_w_k', '/content/10//grigorowich_d_w', '/content/10//dobroljubow_n_a', '/content/10//gorxkij_m', '/content/10//brjusow_w_j', '/content/10//gogolx_n_w', '/content/10//karamzin_n_m', '/content/10//esenin_s_a', '/content/10//grech_n_i']
/content/10//goncharow_i_a/text_0020.shtml.txt
/content/10//goncharow_i_a/text_0010.shtml.txt
/content/10//kjuhelxbeker_w_k/text_0360.shtml.txt
/content/10//kjuhelxbeker_w_k/text_0270.shtml.txt
/content/10//kjuhelxbeker_w_k/text_0242.shtml.txt
/content/10//kjuhelxbeker_w_k/text_0060.shtml.txt
/content/10//kjuhelxbeker_w_k/text_0350.shtml.txt
/content/10//kjuhelxbeker_w_k/text_0210.shtml.txt
/content/10//kjuhelxbeker_w_k/text_1820_evropeiskie_pisma.shtml.txt
/content/10//kjuhelxbeker_w_k/text_0320.shtml.txt
/content/10//kjuhelxbeker_w_k/text_0

In [ ]:
'''
We will also write the texts themselves to the file, without distributions
'''
import csv

path = '/content/10/'

list_names = os.listdir(path)
print(list_names)
roots = []

for name in list_names:
    roots.append(path + '/'+ name)

print(roots)
label = 0 
for t_path in roots:
    label+=1
    for root, dirs, files in os.walk(t_path):
        for filename in files:
                file_path = t_path+'/'+filename
                with open(file_path, 'r', encoding='utf-8') as f:
                  print(file_path)
                  text = f.read().lower()
                  final_data = []
                  final_data.append(str(text)) # write text
                  final_data.insert(0, label) # and a label
                  
                  with open(t_path + ' text' +'.csv', 'a', newline='') as csvfile:
                    writer = csv.writer(csvfile)
  
                    writer.writerow(final_data)
                    csvfile.close()

['goncharow_i_a', 'kjuhelxbeker_w_k', 'grigorowich_d_w', 'dobroljubow_n_a', 'gorxkij_m', 'brjusow_w_j', 'gogolx_n_w', 'karamzin_n_m', 'esenin_s_a', 'grech_n_i']
['/content/10//goncharow_i_a', '/content/10//kjuhelxbeker_w_k', '/content/10//grigorowich_d_w', '/content/10//dobroljubow_n_a', '/content/10//gorxkij_m', '/content/10//brjusow_w_j', '/content/10//gogolx_n_w', '/content/10//karamzin_n_m', '/content/10//esenin_s_a', '/content/10//grech_n_i']
/content/10//goncharow_i_a/text_0020.shtml.txt13.txt
/content/10//goncharow_i_a/text_0020.shtml.txt37.txt
/content/10//goncharow_i_a/text_0020.shtml.txt20.txt
/content/10//goncharow_i_a/text_0010.shtml.txt1.txt
/content/10//goncharow_i_a/text_0020.shtml.txt3.txt
/content/10//goncharow_i_a/text_0020.shtml.txt36.txt
/content/10//goncharow_i_a/text_0010.shtml.txt15.txt
/content/10//goncharow_i_a/text_0020.shtml.txt1.txt
/content/10//goncharow_i_a/text_0020.shtml.txt21.txt
/content/10//goncharow_i_a/text_0020.shtml.txt18.txt
/content/10//goncharo

In [ ]:
'''
get common csv with all authors

'''

import glob
import pandas as pd


path = "/content/authors/content/"

all_files = glob.glob(os.path.join(path, "*features.csv")) # we find files whose name matches the pattern "* features.csv", where * is a part of the name of not specified length 
all_csv = (pd.read_csv(f, sep=',', header=None) for f in all_files)
df_merged   = pd.concat(all_csv, ignore_index=True)
df_merged.to_csv("all_authors.csv") # common csv file for all authors, containing attribute spaces for each text


ValueError: ignored

In [ ]:
'''

file content

'''

df = pd.read_csv('all_authors.csv', skiprows=1, header=None) 
df = df.drop(0, 1)
df


In [ ]:
'''
common csv with texts

'''
import glob
import pandas as pd
path = "/content/10/"

all_files = glob.glob(os.path.join(path, "*text.csv"))
all_csv = (pd.read_csv(f, sep=',', header=None) for f in all_files)
df_merged   = pd.concat(all_csv, ignore_index=True)
df_merged.to_csv("10_texts.csv") # общий csv-файл для всех авторов, сожержащий тексты 


In [ ]:
'''
file content

'''

df = pd.read_csv('10_texts.csv', skiprows=1, header=None)
df = df.drop(0, 1)
df

,1,2
0,7,"всё было готово не позже, как через две недели..."
1,7,﻿\n\nlib.ru/классика: гоголь николай васильеви...
2,7,"душен, как сам петербург. один был человек, на..."
3,7,﻿\n\nlib.ru/классика: гоголь николай васильеви...
4,7,ногими довольно неуместными {глупы} и напыщенн...
...,...,...
2469,4,"логия -- двоих, троих и т. д.-- по числу участ..."
2470,4,рректуры дошли до нашего времени в дефектном с...
2471,4,"\n 2. по-видимому, трактат прудона 18-16 год..."
2472,4,"впервые -- материалы, стр. 501--503.\n 1...."


In [ ]:
# download files

from google.colab import files
#files.download('/content/all_authors.csv') 
files.download('/content/10_texts.csv')   

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>